In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from scipy.optimize import minimize
np.set_printoptions(precision=8, suppress=True)

In [2]:
data = pd.read_csv('C:/Users/91959/Desktop/CODE'
                '/Robust-Penalized-Empirical-Likelihood-Estimation-Method-for-Linear-Regression/Data/Alcohol.csv')

# Independent variables (features) - all columns except the first (Alcohol) and last (ln (Sol)exp)
X = data.iloc[:, 1:-1].values  # Exclude the first column (Alcohol) and the last column (ln (Sol)exp)
# Dependent variable (target) - last column (ln (Sol)exp)
y = data.iloc[:, -1].values

# Robust Parameter Estimation
## M-Estimation Method

In [3]:
# Define Tukey's Biweight Loss Function
def tukey_biweight_loss(residuals, c=4.685):
    """
    Tukey's Biweight loss function.
    c: Tuning constant (default is 4.685 for 95% efficiency under normality).
    """
    t = np.abs(residuals) / c
    loss = np.where(t < 1, (c**2 / 6) * (1 - (1 - t**2)**3), c**2 / 6)
    return loss

# Define the Objective Function for M-Estimation
def m_estimation_objective(beta, X, y, c=4.685):
    """
    Objective function for M-Estimation using Tukey's Biweight loss.
    """
    residuals = y - np.dot(X, beta)  # Residuals
    loss = tukey_biweight_loss(residuals, c)  # Tukey's Biweight loss
    return np.sum(loss)  # Sum of losses

# Set up k-fold cross-validation
k = 5  # Number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=46)

# Lists to store results for M-Estimation
m_est_cv_betas = []
m_est_cv_r2_scores = []

In [4]:
# Perform cross-validation
for fold, (train_idx, test_idx) in enumerate(kf.split(X), 1):
    # Split data for this fold
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Add bias term to training data
    X_train_bias = np.column_stack((np.ones(X_train.shape[0]), X_train))

    # Initial guess for beta (start with OLS estimates)
    XT = X_train_bias.T
    XT_X = np.dot(XT, X_train_bias)
    XT_X_inv = np.linalg.inv(XT_X)
    XT_y = np.dot(XT, y_train)
    beta_initial = np.dot(XT_X_inv, XT_y)

    # Optimize the M-Estimation objective function
    result = minimize(m_estimation_objective, beta_initial, args=(X_train_bias, y_train), method='BFGS')

    # Extract the M-Estimation coefficients
    beta_m_est = result.x

    # Add bias term to test data
    X_test_bias = np.column_stack((np.ones(X_test.shape[0]), X_test))

    # Make predictions
    y_pred = np.dot(X_test_bias, beta_m_est)

    # Calculate R-squared
    y_test_mean = np.mean(y_test)
    SS_res = np.sum((y_test - y_pred) ** 2)
    SS_tot = np.sum((y_test - y_test_mean) ** 2)
    r2 = 1 - (SS_res / SS_tot)

    # Store results
    m_est_cv_betas.append(beta_m_est)
    m_est_cv_r2_scores.append(r2)

In [5]:
# Calculate overall mean and standard deviation of results
m_est_mean_beta = np.mean(m_est_cv_betas, axis=0)
m_est_std_beta = np.std(m_est_cv_betas, axis=0)
m_est_mean_r2 = np.mean(m_est_cv_r2_scores)
m_est_std_r2 = np.std(m_est_cv_r2_scores)

print("\nOverall Results [M-Estimation with Tukey's Biweight]:")
print(f"Mean R-squared: {m_est_mean_r2:.4f} (±{m_est_std_r2:.4f})")
print("\nMean Beta Coefficients:")
feature_names = ['Intercept'] + list(data.columns[1:-1])
for name, beta_mean, beta_std in zip(feature_names, m_est_mean_beta, m_est_std_beta):
    print(f"{name}: {beta_mean:.4f} (±{beta_std:.4f})")


Overall Results [M-Estimation with Tukey's Biweight]:
Mean R-squared: 0.9353 (±0.0605)

Mean Beta Coefficients:
Intercept: 28.5005 (±19.9594)
SAG: -0.0051 (±0.0195)
V: -0.0137 (±0.0208)
Log P: -2.6200 (±0.5230)
P: 26.9565 (±17.4149)
RM: -1.1780 (±1.3020)
Mass: -3.0980 (±2.5931)


## MM-Estimation

In [6]:
# Define Tukey's Biweight Loss Function
def tukey_biweight_loss(residuals, c=4.685):
    """
    Tukey's Biweight loss function.
    c: Tuning constant (default is 4.685 for 95% efficiency under normality).
    """
    t = np.abs(residuals) / c
    loss = np.where(t < 1, (c**2 / 6) * (1 - (1 - t**2)**3), c**2 / 6)
    return loss

# Define the S-Estimation Objective Function
def s_estimation_objective(sigma, residuals, c=4.685):
    """
    Objective function for S-Estimation using Tukey's Biweight loss.
    """
    t = np.abs(residuals) / sigma
    loss = np.where(t < 1, (c**2 / 6) * (1 - (1 - t**2)**3), c**2 / 6)
    return np.sum(loss) - (len(residuals) / 2)  # S-Estimation condition

# Define the M-Estimation Objective Function
def m_estimation_objective(beta, X, y, sigma, c=4.685):
    """
    Objective function for M-Estimation using Tukey's Biweight loss.
    """
    residuals = y - np.dot(X, beta)  # Residuals
    loss = tukey_biweight_loss(residuals / sigma, c)  # Tukey's Biweight loss
    return np.sum(loss)  # Sum of losses

# Set up k-fold cross-validation
k = 5  # Number of folds
kf2 = KFold(n_splits=k, shuffle=True, random_state=46)

# Lists to store results for MM-Estimation
mm_est_cv_betas = []
mm_est_cv_r2_scores = []

In [7]:
# Perform cross-validation
for fold, (train_idx, test_idx) in enumerate(kf2.split(X), 1):
    # Split data for this fold
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Add bias term to training data
    X_train_bias = np.column_stack((np.ones(X_train.shape[0]), X_train))

    # Step 7: S-Estimation - Estimate the scale parameter sigma
    # Initial guess for sigma (use MAD as a robust initial estimate)
    residuals_initial = y_train - np.dot(X_train_bias, np.linalg.lstsq(X_train_bias, y_train, rcond=None)[0])
    sigma_initial = np.median(np.abs(residuals_initial)) / 0.6745  # MAD estimate

    # Optimize the S-Estimation objective function
    result_s = minimize(s_estimation_objective, sigma_initial, args=(residuals_initial), method='BFGS')
    sigma_est = result_s.x[0]  # Estimated scale parameter

    # Step 8: M-Estimation - Estimate the regression coefficients
    # Initial guess for beta (start with OLS estimates)
    XT = X_train_bias.T
    XT_X = np.dot(XT, X_train_bias)
    XT_X_inv = np.linalg.inv(XT_X)
    XT_y = np.dot(XT, y_train)
    beta_initial = np.dot(XT_X_inv, XT_y)

    # Optimize the M-Estimation objective function
    result_m = minimize(m_estimation_objective, beta_initial, args=(X_train_bias, y_train, sigma_est), method='BFGS')
    beta_mm_est = result_m.x  # MM-Estimation coefficients

    # Add bias term to test data
    X_test_bias = np.column_stack((np.ones(X_test.shape[0]), X_test))

    # Make predictions
    y_pred = np.dot(X_test_bias, beta_mm_est)

    # Calculate R-squared
    y_test_mean = np.mean(y_test)
    SS_res = np.sum((y_test - y_pred) ** 2)
    SS_tot = np.sum((y_test - y_test_mean) ** 2)
    r2 = 1 - (SS_res / SS_tot)

    # Store results
    mm_est_cv_betas.append(beta_mm_est)
    mm_est_cv_r2_scores.append(r2)

In [8]:
# Calculate overall mean and standard deviation of results
mm_est_mean_beta = np.mean(mm_est_cv_betas, axis=0)
mm_est_std_beta = np.std(mm_est_cv_betas, axis=0)
mm_est_mean_r2 = np.mean(mm_est_cv_r2_scores)
mm_est_std_r2 = np.std(mm_est_cv_r2_scores)

print("\nOverall Results [MM-Estimation with Tukey's Biweight]:")
print(f"Mean R-squared: {mm_est_mean_r2:.4f} (±{mm_est_std_r2:.4f})")
print("\nMean Beta Coefficients:")
feature_names = ['Intercept'] + list(data.columns[1:-1])
for name, beta_mean, beta_std in zip(feature_names, mm_est_mean_beta, mm_est_std_beta):
    print(f"{name}: {beta_mean:.4f} (±{beta_std:.4f})")


Overall Results [MM-Estimation with Tukey's Biweight]:
Mean R-squared: 0.9355 (±0.0598)

Mean Beta Coefficients:
Intercept: 28.5371 (±19.9718)
SAG: -0.0062 (±0.0195)
V: -0.0129 (±0.0203)
Log P: -2.6110 (±0.5255)
P: 26.8751 (±17.4076)
RM: -1.1528 (±1.3062)
Mass: -3.0969 (±2.5930)


---